In [1]:
import pandas as pd
from utils import Options
import pickle as pkl
import numpy as np
from glob import glob
from readresults.readres_mvpa import merge_results, parse_roi_info, get_subj_avg
import seaborn as sns
from plotting.plot import plot_by_nvoxels

%load_ext autoreload
%autoreload 2

/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
reslist = glob('../../MVPA_results/mainanalysis.csv')

In [3]:
results = merge_results(reslist)

In [51]:
results = parse_roi_info(results)

In [5]:
results.expected.unique()

array([True, False, nan], dtype=object)

In [68]:
results.isna().any()

correct            False
output             False
rawdistance        False
distance           False
chunk              False
view               False
expected            True
subject            False
roi                False
approach           False
traindataformat    False
testdataformat     False
traintask          False
testtask           False
trainmodel         False
testmodel          False
hemi               False
contrast            True
nvoxels             True
dtype: bool

In [69]:
results = results.replace(np.nan, 'none')

In [71]:
results.expected.unique()

array([True, False, 'none'], dtype=object)

In [53]:
ind_vars = ['subject', 'roi', 'approach', 
                'traindataformat', 'testdataformat', 'traintask',
                'testtask', 'trainmodel', 'testmodel', 
                'hemi', 'contrast', 'expected']
ind_vars = [i for i in ind_vars if i in results.columns]
varcombinations = [r.to_dict() for _, r in 
                   results[ind_vars].drop_duplicates().iterrows()]

In [60]:
len(varcombinations)

1680

In [61]:
len(results)

783024

In [66]:
from tqdm import tqdm
for vc in tqdm(varcombinations):
    for i, v in enumerate(vc):
        if i==0:
            thismask = results[v]==vc[v]
        else:
            thismask &= results[v]==vc[v]
    theseresults = results[thismask]
    for nv in theseresults.nvoxels.unique():
        if pd.isna(nv):
            print(nv)

100%|██████████| 1680/1680 [10:34<00:00,  2.65it/s]


In [59]:
varcombinations[0]

{'subject': 'sub-001',
 'roi': 'ba-17-18',
 'approach': 'traintest',
 'traindataformat': 'betas',
 'testdataformat': 'betas',
 'traintask': 'train',
 'testtask': 'test',
 'trainmodel': 5,
 'testmodel': 15,
 'hemi': 'L',
 'contrast': 'objscrvsbas',
 'expected': True}

In [62]:
results.expected.unique()

array([True, False, nan], dtype=object)

In [64]:
results[(results['nvoxels'].isna())&(thismask)]

,correct,output,rawdistance,distance,chunk,view,expected,subject,roi,approach,traindataformat,testdataformat,traintask,testtask,trainmodel,testmodel,hemi,contrast,nvoxels


In [57]:
results[thismask].nvoxels.unique()

array([ 100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,  900.,
       1000., 1100., 1200., 1300., 1400., 1500., 1600., 1700., 1800.,
       1900., 2000., 2100., 2200., 2300., 2400., 2500., 2600., 2700.,
       2800., 2900., 3000.])

In [38]:
thismask = results['subject']=='sub-001'

In [40]:
thismask &= results['testmodel']==15

In [41]:
thismask.sum()

4872

In [24]:
varcombinations

<generator object DataFrame.iterrows at 0x7f0e0866f7b0>

In [18]:
thisresmask = results[f]==varcombinations[0]._fields

('subject',
 'roi',
 'approach',
 'traindataformat',
 'testdataformat',
 'traintask',
 'testtask',
 'trainmodel',
 'testmodel',
 'expected')

In [19]:
varcombinations['subject']

TypeError: list indices must be integers or slices, not str

In [6]:
avgres = get_subj_avg(results, avg_decodedirs=True)

In [7]:
avgres = parse_roi_info(avgres)

In [8]:
avgres.trainmodel.unique()

array(['5_15', '5_17'], dtype=object)

In [9]:
avgres.roi.unique()

array(['LO', 'ba-17-18'], dtype=object)

In [10]:
EVCdata = avgres[avgres['roi']=='ba-17-18']
LOCdata = avgres[avgres['roi']=='LO']

In [11]:
EVCdata.nvoxels.unique()

array([  nan,  100., 1000., 1100., 1200., 1300., 1400., 1500., 1600.,
       1700., 1800., 1900.,  200., 2000., 2100., 2200., 2300., 2400.,
       2500., 2600., 2700., 2800., 2900.,  300., 3000.,  400.,  500.,
        600.,  700.,  800.,  900.])

In [12]:
EVCdata.head()

,subject,roi,approach,expected,correct,output,rawdistance,distance,trainmodel,testmodel,traintask,testtask,hemi,contrast,nvoxels
75,sub-001,ba-17-18,traintest,False,0.556818,0.534091,0.029835,0.134367,5_15,5_15,train_test,train_test,L,None,NaN
76,sub-001,ba-17-18,traintest,True,0.666667,0.507576,0.029824,0.349284,5_15,5_15,train_test,train_test,L,None,NaN
77,sub-001,ba-17-18,traintest,NaN,0.681818,0.522727,0.029835,0.485971,5_15,5_15,train_test,train_test,L,None,NaN
78,sub-001,ba-17-18,traintest,False,0.500000,0.522727,-0.168055,0.094735,5_15,5_15,train_test,train_test,L,objscrvsbas,100.0
79,sub-001,ba-17-18,traintest,True,0.647727,0.511364,-0.005755,0.311978,5_15,5_15,train_test,train_test,L,objscrvsbas,100.0


In [15]:
plot_by_nvoxels(EVCdata)

> /project/3018040.05/rotscenetask_fmri/analysis/plotting/plot.py(134)df_to_array_tfce()
    133     for i, sub in enumerate(np.sort(df.subject.unique())):
--> 134         for j, nv in enumerate(np.sort(df.nvoxels.unique())):
    135             thisdata = df[(df['subject']==sub)&(df['nvoxels']==nv)]



ipdb>  thisdata[thisdata['expected']==True][measure].values


array([], dtype=float64)


ipdb>  thisdata


Empty DataFrame
Columns: [subject, nvoxels, expected, correct, output, rawdistance, distance]
Index: []


...

'sub-008'


ipdb>  nv


2900.0


ipdb>  q


BdbQuit: 

In [ ]:
EVCdata[EVCdata['trainmodel']=='5_17'].correct.mean()

In [ ]:
LOCdata[LOCdata['trainmodel']=='5_17'].correct.mean()

In [ ]:
avgres

In [ ]:
results[results['approach']=='CV'].chunk.unique()

In [ ]:
from readresults.readres_mvpa import parse_roi_info

In [ ]:
results_avg = results.groupby(['subject', 'roi']).mean().reset_index()

In [ ]:
results = parse_roi_info(results)

In [ ]:
results = results.drop(['chunk'], axis=1)

In [ ]:
ind_vars = ['subject', 'roi', 'approach', 
            'traindataformat', 'testdataformat', 'traintask',
            'testtask', 'trainmodel', 'testmodel', 
            'hemi', 'contrast', 'nvoxels']
ind_vars = [i for i in ind_vars if i in results.columns and results[i].nunique() > 0]

In [ ]:
results_avg = results.groupby(ind_vars).mean().reset_index()

In [ ]:
results_avg = results_avg.drop(['chunk'], axis=1)

In [ ]:
results_avg.head()

In [ ]:
sns.barplot(data=results_avg, x='roi', y='distance')

In [ ]:
allinfo = 'LO_contr-objscrvsbas_top-1000'.split('_')

In [ ]:
allinfo[1] in ['L', 'R']

In [ ]:
allinfo

In [ ]:
contrindx = 1

In [ ]:
allinfo[contrindx].split('contr-')[1]

In [ ]:
a = [None, None, None]
all(v is None for v in a)

In [ ]:
sns.barplot(data=results, x='roi', y='correct')

In [ ]:
results[results['roi'].isin(['rBA17_18_L', 'rBA17_18_R',
                             'ba-17-18_L', 'ba-17-18_R'])].distance.mean()

In [ ]:
outputs = [0.3, 4.0, 6.9, 2.0, 3.5, 0.1]
targets = [4.7, 3.2, 1.0, 2.9, 3.2, 6.8]
splits = [1, 1, 1, 2, 2, 2]
example_res = pd.DataFrame(
    list(zip(outputs, targets, splits)),
    columns=['output', 'target', 'split'])

In [ ]:
example_res

In [ ]:
x = np.array([3, 4, 5])
np.isin(x, [5])

In [ ]:
example_res = np.nan
isinstance(example_res, float) and pd.isnull(example_res)

In [ ]:
pd.isna(example_res).all()

In [ ]:
outs2 = [0.5, 0.65, 1.2]
splits2 = [3, 3, 3]
example_res_2 = pd.DataFrame(
    list(zip(outs2, splits2)),
    columns=['output', 'split'])

In [ ]:
all_res = pd.concat([example_res, example_res_2])

In [ ]:
all_res

In [ ]:
for s in sorted(example_res.split.unique()):
    thissplitlength = len(example_res[example_res['split']==s])
    example_res.loc[example_res['split']==s, 'sample'] = list(range(thissplitlength))

In [ ]:
groupedres = example_res.groupby('sample').mean().reset_index().drop(['sample', 'split'], axis=1)

In [ ]:
groupedres

In [ ]:
agg_functions = {'output': 'mean', 'target': 'sum'}

#create new DataFrame by combining rows with same id values
res_new = example_res.groupby(example_res['split']).aggregate(
    agg_functions)

In [ ]:
res_new

In [ ]:
with open('/project/3018040.05/MVPA_results/traintest.pkl', 'rb') as f:
    results = pkl.load(f)

In [ ]:
res = results['train-trainm3-test-testm3_ba-17-18_L_betas'][0]
accuracy = res[:, 0]==res[:, 1]
res_df = pd.DataFrame(res, 
                      columns = ['output', 'target', 'distance', 'chunk'])

In [ ]:
res_df['trialno'] = 1

In [ ]:
res_df.to_numpy()

In [ ]:
accuracy.shape

In [ ]:
random = np.random.rand(accuracy.shape[0])

In [ ]:
random

In [ ]:
output = (res[:, 0] == np.unique(res[:, 1])[0]).astype(int)

In [ ]:
output

In [ ]:
np.mean([True, True, False])

In [ ]:
outputs = 

In [ ]:
res_df.distance.dtype

In [ ]:
results['train-trainm3-test-testm3_ba-17-18_L_betas'][0].shape

In [ ]:
opt = Options()

In [ ]:
import configs

In [ ]:
dir(configs)

In [ ]:
configs.mvpa_outdir

In [ ]:
from configs import mvpa_outdir as data_dir

In [ ]:
from six.moves import cPickle as pickle

In [ ]:
data_dir

In [ ]:
mvpa_datadir

In [ ]:
x

In [ ]:
opt.sub = 'sub-001'

In [ ]:
opt.sub

In [ ]:
eventsfile = '/project/3018040.05/bids/sub-001/func/sub-001_task-test_run-1_events.tsv'
behavfile = '/project/3018040.05/bids/sub-001/func/sub-001_task-test_beh.tsv'
events = pd.read_csv(eventsfile, sep='\t')
behav = pd.read_csv(behavfile, sep='\t')

In [ ]:
events

In [ ]:
bedscenes = [2, 5, 6, 7, 9, 13, 14, 17, 18, 20]
bedindx = behav.index[behav['Scene'].isin(bedscenes)]
couchindx = behav.index[~behav['Scene'].isin(bedscenes)]

In [ ]:
bedindx

In [ ]:
couchindx